In [22]:
import numpy as np
import pandas as pd

## Задача 11
- Загрузите данные 03-1-airline-passengers.csv.

In [57]:
df = pd.read_csv(
    '../data/03-1-airline-passengers.csv', 
    names=['month', 'value'], 
    parse_dates=['month'], 
    index_col='month',
    header=0
    )

df.head()

,value
month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


- Создайте признаки lag_1 и lag_12 (значение за предыдущий месяц и за 12 месяцев назад).Выведите первые 5 строк с новыми признаками.

In [58]:
df['lag_1'] = df['value'].diff(1)
df['lag_12'] = df['value'].diff(12)

- Создайте скользящие признаки:
    - rolling_mean_3 — среднее за последние 3 месяца
    - rolling_std_3 — стандартное отклонение за последние 3 месяца
    - Выведите первые 5 строк с новыми признаками.

In [59]:
df['rolling_mean_3'] = df['value'].rolling(3).mean()
df['rolling_std_3'] = df['value'].rolling(3).std()

df.head()

,value,lag_1,lag_12,rolling_mean_3,rolling_std_3
month,,,,,
1949-01-01,112,NaN,NaN,NaN,NaN
1949-02-01,118,6.0,NaN,NaN,NaN
1949-03-01,132,14.0,NaN,120.666667,10.263203
1949-04-01,129,-3.0,NaN,126.333333,7.371115
1949-05-01,121,-8.0,NaN,127.333333,5.686241


- Создайте признак дифференцирования: разность текущего значения и предыдущего (diff). Выведите первые 5 строк с новым признаком. 

In [60]:
df['diff_lag_1'] = df['value'] - df['lag_1']
df.head()

,value,lag_1,lag_12,rolling_mean_3,rolling_std_3,diff_lag_1
month,,,,,,
1949-01-01,112,NaN,NaN,NaN,NaN,NaN
1949-02-01,118,6.0,NaN,NaN,NaN,112.0
1949-03-01,132,14.0,NaN,120.666667,10.263203,118.0
1949-04-01,129,-3.0,NaN,126.333333,7.371115,132.0
1949-05-01,121,-8.0,NaN,127.333333,5.686241,129.0


- Добавьте календарные признаки: день недели (dayofweek), месяц (month) и признак выходного дня (is_weekend). Выведите первые 5 строк.

In [61]:
df['month'] = df.index.month
df['dayofweek'] = df.index.dayofweek
df['is_weekend'] = df.index.dayofweek.isin([5, 6])

df.head()

,value,lag_1,lag_12,rolling_mean_3,rolling_std_3,diff_lag_1,month,dayofweek,is_weekend
month,,,,,,,,,
1949-01-01,112,NaN,NaN,NaN,NaN,NaN,1,5,True
1949-02-01,118,6.0,NaN,NaN,NaN,112.0,2,1,False
1949-03-01,132,14.0,NaN,120.666667,10.263203,118.0,3,1,False
1949-04-01,129,-3.0,NaN,126.333333,7.371115,132.0,4,4,False
1949-05-01,121,-8.0,NaN,127.333333,5.686241,129.0,5,6,True


- Добавьте признак тренда — возрастающий счётчик по времени (trend). Выведите первые 5 строк.

In [62]:
df['trend'] = np.arange(len(df))

df.head()

,value,lag_1,lag_12,rolling_mean_3,rolling_std_3,diff_lag_1,month,dayofweek,is_weekend,trend
month,,,,,,,,,,
1949-01-01,112,NaN,NaN,NaN,NaN,NaN,1,5,True,0
1949-02-01,118,6.0,NaN,NaN,NaN,112.0,2,1,False,1
1949-03-01,132,14.0,NaN,120.666667,10.263203,118.0,3,1,False,2
1949-04-01,129,-3.0,NaN,126.333333,7.371115,132.0,4,4,False,3
1949-05-01,121,-8.0,NaN,127.333333,5.686241,129.0,5,6,True,4


- Объедините все созданные признаки (лаги, rolling, diff, календарные признаки, тренд) в один датафрейм. Удалите строки с пропусками, которые образовались из-за лагов и rolling окон. Выведите размер итогового датафрейма.

In [68]:
df.dropna(inplace=True)

df.shape

(132, 10)

- Обучите простую модель линейной регрессии на признаках, созданных в предыдущем задании, чтобы предсказать значение временного ряда на следующий месяц. 
    - Выведите коэффициенты модели.
    - Реализуйте отбор признаков с помощью L1-регуляризации (Lasso).
    - Выведите признаки с ненулевыми коэффициентами.

In [69]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [73]:
X = df.drop('value', axis=1)    # признаки
y = df['value']                 # целевая переменная

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (важно для регуляризации)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Простая линейная регрессия
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

LinearRegression()

In [74]:
print("Коэффициенты линейной регрессии:")
for feature, coef in zip(X.columns, lr.coef_):
    print(f"{feature}: {coef:.4f}")
print(f"Intercept: {lr.intercept_:.4f}")

Коэффициенты линейной регрессии:
lag_1: 34.8220
lag_12: -0.0000
rolling_mean_3: 0.0000
rolling_std_3: -0.0000
diff_lag_1: 113.1995
month: 0.0000
dayofweek: 0.0000
is_weekend: -0.0000
trend: 0.0000
Intercept: 298.2000


In [75]:
#  Lasso регрессия для отбора признаков
lasso = Lasso(alpha=0.1, random_state=42)  # alpha - сила регуляризации
lasso.fit(X_train_scaled, y_train)

# Признаки с ненулевыми коэффициентами
non_zero_coefs = lasso.coef_ != 0
selected_features = X.columns[non_zero_coefs]
selected_coefs = lasso.coef_[non_zero_coefs]

print("\nПризнаки с ненулевыми коэффициентами (Lasso):")
for feature, coef in zip(selected_features, selected_coefs):
    print(f"{feature}: {coef:.4f}")
print(f"Intercept: {lasso.intercept_:.4f}")
print(f"Количество отобранных признаков: {sum(non_zero_coefs)}")


Признаки с ненулевыми коэффициентами (Lasso):
lag_1: 34.5857
rolling_mean_3: 1.8338
diff_lag_1: 111.2587
Intercept: 298.2000
Количество отобранных признаков: 3
